# Create a FABRIC Facility Port

This notebook shows how to use create an facility port to connect your FABRIC experiment to an external facility. 



## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
Bastion Username,marcos_schwarz_0034669026
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [2]:
slice_name = "MySlice"

facility_port='AmLight-EXP-Layer2-FIU'
facility_port_site='FIU'
facility_port_vlan='4000'

#Create a slice
slice = fablib.new_slice(name=slice_name)

node = slice.add_node(name=f"Node1", site=facility_port_site)

node.add_post_boot_upload_directory('node_tools','.')
node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }}')
node.add_post_boot_execute('sudo node_tools/install.sh')

#node_iface = node.add_component(model='NIC_ConnectX_6', name="nic1").get_interfaces()[0]
node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

facility_port = slice.add_facility_port(name=facility_port, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface =facility_port.get_interfaces()[0]

print(f"facility_port.get_site(): {facility_port.get_site()}")

#net = slice.add_l2network(name=f'net_facility_port', interfaces=[node_iface,facility_port_interface])
net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

#Submit the Request
slice.submit();


Retry: 11, Time: 317 sec


ID,814285c6-1a3b-4ad2-956a-aa002515bda7
Name,MySlice
Lease Expiration (UTC),2023-11-14 23:46:21 +0000
Lease Start (UTC),2023-11-13 23:46:21 +0000
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
9ab3a506-4842-4940-8337-6683a501fefa,Node1,2,8,10,default_rocky_8,qcow2,fiu-w2.fabric-testbed.net,FIU,rocky,131.94.57.43,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.43,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
94e9f642-e850-4e57-9d50-4e616d6b7778,net_facility_port,L2,L2Bridge,FIU,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,02:59:B4:EA:50:7E,eth1,eth1,fe80::45e5:fb49:b4a7:b9ae,6



Time to print interfaces 318 seconds


## Observe the Slice's Attributes


In [6]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


ID,814285c6-1a3b-4ad2-956a-aa002515bda7
Name,MySlice
Lease Expiration (UTC),2023-11-14 23:46:21 +0000
Lease Start (UTC),2023-11-13 23:46:21 +0000
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
9ab3a506-4842-4940-8337-6683a501fefa,Node1,2,8,10,default_rocky_8,qcow2,fiu-w2.fabric-testbed.net,FIU,rocky,131.94.57.43,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.43,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
94e9f642-e850-4e57-9d50-4e616d6b7778,net_facility_port,L2,L2Bridge,FIU,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,02:59:B4:EA:50:7E,eth1,eth1,10.40.0.101,6


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,02:59:B4:EA:50:7E,eth1,eth1,10.40.0.101,6


## Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [4]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("10.40.0.0/24")
available_ips = list(subnet)[2:]

In [5]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')


node1_addr: 10.40.0.101
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:59:b4:ea:50:7e brd ff:ff:ff:ff:ff:ff
    inet 10.40.0.101/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::45e5:fb49:b4a7:b9ae/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [ ]:
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

stdout, stderr = node1.execute(f'ping -c 5 10.40.0.10')

## Configure Slice Parameters


In [5]:
slice_name='MySlice'

## Get and Renew the Slice

You slice is in the list of all your slices. You can loop through the list of slices to get the slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

The new end date is in UTC.

In [8]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [10]:
slice.show()

ID,814285c6-1a3b-4ad2-956a-aa002515bda7
Name,MySlice
Lease Expiration (UTC),2023-11-20 21:54:11 +0000
Lease Start (UTC),2023-11-13 23:46:21 +0000
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
State,StableOK


ID,814285c6-1a3b-4ad2-956a-aa002515bda7
Name,MySlice
Lease Expiration (UTC),2023-11-20 21:54:11 +0000
Lease Start (UTC),2023-11-13 23:46:21 +0000
Project ID,f1f5b0f8-0eef-40f3-940e-c4c8ff2d60ca
State,StableOK


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [4]:
slice.delete()